In [1]:
!{__import__('sys').executable} -m pip install --quiet --upgrade -r requirements.txt

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbgrader 0.6.2 requires traitlets==4.3.3, but you have traitlets 5.1.0 which is incompatible.


In [4]:
session_storage = 'downloads'

import os
os.makedirs(session_storage, exist_ok=True)

In [5]:
import fsspec

fs_dataset = fsspec.filesystem(
    'simplecache', 
    target_protocol='https', 
    cache_storage=session_storage,
)

In [ ]:

# https://www.diva-portal.org/smash/get/diva2:1571821/FULLTEXT02

# https://keras.io/examples/vision/attention_mil_classification/
# https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [66]:
#from urllib.parse import urlparse
#import os
#os.path.split(
#    urlparse('/Movie Scripts/Reservoir Dogs Script.html').path
#)
#len(list(filter(lambda s: s.endswith('.html'), get_movie_links())))

('/Movie Scripts', 'Reservoir Dogs Script.html')

In [3]:
import requests
import lxml
import lxml.html

import os
import urllib


def get_all_hrefs(url):
    return map(
        lambda s: urllib.parse.urlparse(s).path,
        lxml.html.fromstring(
            requests.get(url).text
        ).xpath('//a/@href')
    )


import joblib

def get_script(script_link):
    res = ''
    for el in lxml.html.fromstring(
        requests.get('https://imsdb.com/' + script_link).text
    ).xpath('//td[@class="scrtext"]/pre'):    
        res += ''.join(el.itertext())
    return res    

def get_movie_links():
    return set(
        filter(
            lambda u: os.path.dirname(u) == '/Movie Scripts',
            get_all_hrefs('https://imsdb.com/all-scripts.html')
        )
    )

def get_script_links(movie_link):
    return set(
        filter(
            lambda u: os.path.dirname(u) == '/scripts',            
            get_all_hrefs('https://imsdb.com/' + movie_link)
        )
    )
    
def get_movie_name(movie_link):
    name, _ = os.path.splitext(
        os.path.basename(movie_link)
    )
    return name
    
def get_movie_script_links():
    def _f(movie_link):
        return (
            get_movie_name(movie_link),
            get_script_links(movie_link)
        )
    return joblib.Parallel(n_jobs=-1)(
        joblib.delayed(_f)(movie_link) for movie_link in get_movie_links()
    )

def get_scripts(movie_script_links):
    def _f(movie_name, script_link):
        return (
            movie_name,
            list(map(
                get_script,
                script_link
            ))
        )

    return joblib.Parallel(n_jobs=-1)(
        joblib.delayed(_f)(movie_name, script_link) 
            for movie_name, script_link in movie_script_links
    )


In [7]:
import pandas as pd

df = pd.DataFrame(
    get_scripts(
        get_movie_script_links()
    ),
    columns=['script_name', 'script_text']
)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   script_name  1209 non-null   object
 1   script_text  1209 non-null   object
dtypes: object(2)
memory usage: 19.0+ KB


In [9]:
df.to_pickle('./dataset.pkl', compression='gzip')

In [10]:
pd.read_pickle('./dataset.pkl', compression='gzip')

,script_name,script_text
0,American Milkshake Script,[\r\n\r\n\r\n\r\n ...
1,"Man Who Knew Too Much, The Script",[ THE MAN WHO KNEW TOO MUCH\...
2,"Crow Salvation, The Script","[\r\n\r\n ""THE CROW: SALVATION"" -- by Chi..."
3,Die Hard 2 Script,[\r\n\r\n \r\n ...
4,Game 6 Script,[ \r\n \r\n \r\n ...
...,...,...
1204,"Cooler, The Script",[\r\n\r\n\r\n ...
1205,Hitchcock Script,[\r\n\r\n\r\n\r\n ...
1206,Breaking Away Script,"[\r\n\r\n \r\n ""BAM..."
1207,Batman Returns Script,[]


In [29]:
#print(df.iloc[1]['script_text'][0])

In [9]:
import pandas as pd

In [3]:
str('Ace-Ventura-Pet-Detective.html').replace('.html', '')

'Ace-Ventura-Pet-Detective'

In [96]:
def get_movie_details(movie_links):
    pass

